In [ ]:
!pip install pyterrier
!pip install python-terrier
!pip install sentence_transformers

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np


In [ ]:
import pyterrier as pt
if not pt.started():
    pt.init()

<ipython-input-78-a9775fa13689>:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


In [ ]:
df_q_explor= pd.read_csv("/content/drive/MyDrive/data_colab/inex_data/query_resource2005.tsv", sep='\t')
df_q_explor['qid'] = df_q_explor['qid'].astype(str)
print(len(df_q_explor.index))

9


In [ ]:
df_q_explor

,qid,query,intent_fin
0,205,marshall mcluhan,resource
1,232,dempster shafer theory database experiment\n,resource
2,234,call for papers conference workshop multimedia,resource
3,239,quantum computation\n,resource
4,243,schafer anand mulvenna riecken,resource
5,264,machine learning mutual information criterion,resource
6,277,find citations to baezayates work,resource
7,282,machine learning,resource
8,283,mutual information criterion,resource


In [ ]:
#remove line breaks and extra spaces
def clean_query(query):
    return ' '.join(query.replace('\n', ' ').split())
df_q_explor['query'] = df_q_explor['query'].apply(clean_query)

In [ ]:
df_q_explor

,qid,query,intent_fin
0,205,marshall mcluhan,resource
1,232,dempster shafer theory database experiment,resource
2,234,call for papers conference workshop multimedia,resource
3,239,quantum computation,resource
4,243,schafer anand mulvenna riecken,resource
5,264,machine learning mutual information criterion,resource
6,277,find citations to baezayates work,resource
7,282,machine learning,resource
8,283,mutual information criterion,resource


In [ ]:
index = pt.IndexFactory.of("/content/drive/MyDrive/data_colab/inex_index/sections_no_sw_2005")

In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=1500)


bm25_res = bm25.transform(df_q_explor)


# Retrieve the TF-IDF scores
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", num_results=1500)
tfidf_res = tfidf.transform(df_q_explor)
tfidf_res

combined_results = pd.merge(bm25_res, tfidf_res[['qid', 'docid', 'score']], on=['qid', 'docid'], suffixes=('_bm25', '_tfidf'))

combined_results

<ipython-input-84-549a86a883bd>:5: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=1500)
<ipython-input-84-549a86a883bd>:16: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", num_results=1500)


,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf
0,205,30819,co/1996/r8091/article[1]/bdy[1]/sec[1],0,30.697313,marshall mcluhan,resource,16.782770
1,205,5940,mu/1996/u2003/article[1]/bdy[1]/sec[1],1,30.196907,marshall mcluhan,resource,16.509190
2,205,2480,pd/1998/p1013/article[1]/bdy[1]/sec[6],2,29.664973,marshall mcluhan,resource,16.218372
3,205,10321,mi/1996/m6054/article[1]/bdy[1]/sec[7],3,28.567031,marshall mcluhan,resource,15.618107
4,205,44457,cs/1999/c6001/article[1]/bdy[1]/sec[1],4,28.463019,marshall mcluhan,resource,15.563131
...,...,...,...,...,...,...,...,...
10306,283,58728,tp/2002/i0881/article[1]/bdy[1]/sec[2],1493,6.501106,mutual information criterion,resource,3.595613
10307,283,78018,so/1997/s2107/article[1]/bdy[1]/sec[3],1494,6.501106,mutual information criterion,resource,3.595613
10308,283,16501,cg/2000/g5076/article[1]/bdy[1]/sec[5],1495,6.498252,mutual information criterion,resource,3.594035
10309,283,15320,cg/1995/g1032/article[1]/bdy[1]/sec[3],1498,6.492070,mutual information criterion,resource,3.590615


In [ ]:
import pandas as pd

df_th = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/thorough_no_empty_no_sw05_corr.tsv',sep="\t")
combined_results = pd.merge(combined_results, df_th[['docno', 'text']], on='docno', how='left')
combined_results

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text
0,205,30819,co/1996/r8091/article[1]/bdy[1]/sec[1],0,30.697313,marshall mcluhan,resource,16.782770,"pervasive medium age television, radio, even p..."
1,205,5940,mu/1996/u2003/article[1]/bdy[1]/sec[1],1,30.196907,marshall mcluhan,resource,16.509190,Marshall McLuhan imagined world future global ...
2,205,2480,pd/1998/p1013/article[1]/bdy[1]/sec[6],2,29.664973,marshall mcluhan,resource,16.218372,"SEEDS TAKING ROOT Although 250,000 individuals..."
3,205,10321,mi/1996/m6054/article[1]/bdy[1]/sec[7],3,28.567031,marshall mcluhan,resource,15.618107,Community Probably microprocessor's greatest s...
4,205,44457,cs/1999/c6001/article[1]/bdy[1]/sec[1],4,28.463019,marshall mcluhan,resource,15.563131,Marshall McLuhan observed technology extension...
...,...,...,...,...,...,...,...,...,...
10306,283,58728,tp/2002/i0881/article[1]/bdy[1]/sec[2],1493,6.501106,mutual information criterion,resource,3.595613,"Filtering Algorithm section, describe filterin..."
10307,283,78018,so/1997/s2107/article[1]/bdy[1]/sec[3],1494,6.501106,mutual information criterion,resource,3.595613,SELECTING IMPLEMENTING INTEGRATION survey exis...
10308,283,16501,cg/2000/g5076/article[1]/bdy[1]/sec[5],1495,6.498252,mutual information criterion,resource,3.594035,ASPECT RATIOS FILMS Hang second. Figure 14 sho...
10309,283,15320,cg/1995/g1032/article[1]/bdy[1]/sec[3],1498,6.492070,mutual information criterion,resource,3.590615,Random Iteration Algorithm Random Iteration Al...


1.INTENT AGNOSTIC FEATURES

In [ ]:
# Calculate the sum, min, max, mean, and variance of the 'score_tfidf' for each query (grouped by 'qid')
tfidf_stats = combined_results.groupby('qid')['score_tfidf'].agg(
    sum_tfidf='sum',
    min_tfidf='min',
    max_tfidf='max',
    mean_tfidf='mean',
    var_tfidf='var'
).reset_index()

tfidf_stats

,qid,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf
0,205,1279.258178,0.475768,16.782770,5.076421,8.202943
1,232,8220.834162,4.725083,24.878222,5.970105,4.931272
2,234,13622.268799,6.818037,18.002224,9.479658,4.249111
3,239,5788.634092,1.871229,10.145334,3.869408,6.259412
4,243,523.640135,1.133187,12.831915,6.308917,6.577692
5,264,8488.602355,4.940182,14.174346,6.128955,1.264033
6,277,5885.310069,3.599292,14.100381,4.478927,2.373837
7,282,7305.245355,3.817447,7.357007,4.969555,0.644630
8,283,6800.525714,3.590080,10.722262,4.542769,0.727339


In [ ]:
combined_results = pd.merge(combined_results, tfidf_stats, on='qid')

In [ ]:
# Download stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
combined_results['query'] = combined_results['query'].astype(str)
combined_results['text'] = combined_results['text'].astype(str)

In [ ]:
stop_words = set(stopwords.words('english'))

# Preprocess the text (tokenization, stopword removal)
def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Apply preprocessing to both queries and documents
combined_results['query_tokens'] = combined_results['query'].apply(preprocess)
combined_results['document_tokens'] = combined_results['text'].apply(preprocess)

# Combine queries and documents into one list for further processing
combined_text = pd.concat([combined_results['query_tokens'].apply(' '.join), combined_results['document_tokens'].apply(' '.join)], axis=0)


In [ ]:
combined_results['document_length'] = combined_results['text'].str.split().str.len()

In [ ]:
combined_results

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf,query_tokens,document_tokens,document_length
0,205,30819,co/1996/r8091/article[1]/bdy[1]/sec[1],0,30.697313,marshall mcluhan,resource,16.782770,"pervasive medium age television, radio, even p...",1279.258178,0.475768,16.782770,5.076421,8.202943,"[marshall, mcluhan]","[pervasive, medium, age, television, radio, ev...",97
1,205,5940,mu/1996/u2003/article[1]/bdy[1]/sec[1],1,30.196907,marshall mcluhan,resource,16.509190,Marshall McLuhan imagined world future global ...,1279.258178,0.475768,16.782770,5.076421,8.202943,"[marshall, mcluhan]","[marshall, mcluhan, imagined, world, future, g...",102
2,205,2480,pd/1998/p1013/article[1]/bdy[1]/sec[6],2,29.664973,marshall mcluhan,resource,16.218372,"SEEDS TAKING ROOT Although 250,000 individuals...",1279.258178,0.475768,16.782770,5.076421,8.202943,"[marshall, mcluhan]","[seeds, taking, root, although, individuals, v...",120
3,205,10321,mi/1996/m6054/article[1]/bdy[1]/sec[7],3,28.567031,marshall mcluhan,resource,15.618107,Community Probably microprocessor's greatest s...,1279.258178,0.475768,16.782770,5.076421,8.202943,"[marshall, mcluhan]","[community, probably, microprocessor, greatest...",134
4,205,44457,cs/1999/c6001/article[1]/bdy[1]/sec[1],4,28.463019,marshall mcluhan,resource,15.563131,Marshall McLuhan observed technology extension...,1279.258178,0.475768,16.782770,5.076421,8.202943,"[marshall, mcluhan]","[marshall, mcluhan, observed, technology, exte...",263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10306,283,58728,tp/2002/i0881/article[1]/bdy[1]/sec[2],1493,6.501106,mutual information criterion,resource,3.595613,"Filtering Algorithm section, describe filterin...",6800.525714,3.590080,10.722262,4.542769,0.727339,"[mutual, information, criterion]","[filtering, algorithm, section, describe, filt...",204
10307,283,78018,so/1997/s2107/article[1]/bdy[1]/sec[3],1494,6.501106,mutual information criterion,resource,3.595613,SELECTING IMPLEMENTING INTEGRATION survey exis...,6800.525714,3.590080,10.722262,4.542769,0.727339,"[mutual, information, criterion]","[selecting, implementing, integration, survey,...",204
10308,283,16501,cg/2000/g5076/article[1]/bdy[1]/sec[5],1495,6.498252,mutual information criterion,resource,3.594035,ASPECT RATIOS FILMS Hang second. Figure 14 sho...,6800.525714,3.590080,10.722262,4.542769,0.727339,"[mutual, information, criterion]","[aspect, ratios, films, hang, second, figure, ...",1001
10309,283,15320,cg/1995/g1032/article[1]/bdy[1]/sec[3],1498,6.492070,mutual information criterion,resource,3.590615,Random Iteration Algorithm Random Iteration Al...,6800.525714,3.590080,10.722262,4.542769,0.727339,"[mutual, information, criterion]","[random, iteration, algorithm, random, iterati...",228


In [ ]:
def all_words_in_text(query, text):
    query_words = set(query.lower().split())  # Tokenizing query and converting to lowercase
    text_words = set(text.lower().split())    # Tokenizing text and converting to lowercase
    return 1 if query_words.issubset(text_words) else 0  # Return 1 if all query words are found, else 0


combined_results['query_in_text'] = combined_results.apply(lambda row: all_words_in_text(row['query'], row['text']), axis=1)


2.INTENT-AWARE FEATURES

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
import numpy as np

# Calculate n-gram jaccard and cosine similarity

def get_ngrams_for_both(query, text, n=2):
    """
    Generate n-grams for both query and text using the same vocabulary.
    """
    vectorizer = CountVectorizer(ngram_range=(n, n), analyzer='word')

    # Fit the vectorizer on both the query and the text to ensure the same vocabulary
    combined_text = [query, text]
    ngrams = vectorizer.fit_transform(combined_text).toarray()

    # Separate the transformed query and text n-grams
    query_ngrams = ngrams[0].reshape(1, -1)  # First row is the query n-grams
    text_ngrams = ngrams[1].reshape(1, -1)   # Second row is the text n-grams

    return query_ngrams, text_ngrams

def calculate_cosine(query, text, n=2):
    """
    Calculate the cosine similarity between n-grams of two strings.
    """
    query_ngrams, text_ngrams = get_ngrams_for_both(query, text, n)

    # Calculate cosine similarity between n-grams
    return cosine_similarity(query_ngrams, text_ngrams)[0][0]

def calculate_jaccard(query, text, n=2):
    """
    Calculate the Jaccard similarity between n-grams of two strings.
    """
    query_ngrams, text_ngrams = get_ngrams_for_both(query, text, n)

    # Convert n-grams to binary (presence/absence of n-grams)
    query_ngrams_binary = (query_ngrams > 0).astype(int)
    text_ngrams_binary = (text_ngrams > 0).astype(int)

    # Calculate Jaccard similarity between binary n-grams
    return jaccard_score(query_ngrams_binary.flatten(), text_ngrams_binary.flatten())


In [ ]:
# Function to check condition and calculate Jaccard or Cosine similarity or return 0
def calculate_similarity_for_row(row, n=2):
    query = row['query']
    text = row['text']

    # Calculate Jaccard similarity
    jaccard_sim = calculate_jaccard(query, text, n)

    # Calculate Cosine similarity
    cosine_sim = calculate_cosine(query, text, n)

    return jaccard_sim, cosine_sim


# Apply the function to each row in the DataFrame
combined_results[['jaccard_similarity', 'cosine_similarity']] = combined_results.apply(lambda row: calculate_similarity_for_row(row, n=2), axis=1, result_type='expand')


In [ ]:
import re


# Check if docno ends with specific patterns
def docno_endswith_conditions(docno):
    #Check if docno ends with introduction
    if docno.endswith('sec[1]'):
        return True
    else:
        return False


def calculate_similarity_for_row2(row, n=2):
    docno = row['docno']
    query = row['query']
    text = row['text']


    if docno_endswith_conditions(docno):  # Check if it is an introduction
        # Calculate Jaccard similarity
        jaccard_sim = calculate_jaccard(query, text, n)
        # Calculate Cosine similarity
        cosine_sim = calculate_cosine(query, text, n)
        return jaccard_sim, cosine_sim
    else:
        return 0, 0  # Return 0 for both Jaccard and Cosine if none of the conditions are met

combined_results[['jaccard_similarity_se', 'cosine_similarity_se']] = combined_results.apply(lambda row: calculate_similarity_for_row2(row, n=2), axis=1, result_type='expand')


In [ ]:
combined_results['exact_match'] = combined_results.apply(lambda row: 1 if row['query'].strip().lower() in row['text'].strip().lower() else 0, axis=1)

In [ ]:
combined_results = combined_results.drop(columns=['query_tokens', 'document_tokens'])

In [ ]:
combined_results.to_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_navig_2005_features_and_sections.tsv', sep='\t', index=False)